Vamos a crear este script capaz de pasarle todas las imagenes al API de clarifi para sacar sus tags o categorias y luego traducirlas al español

In [16]:
!pip install pymongo==4.5.0 clarifai-grpc googletrans==2.4.0 requests translate

Este código arregla el problema de dependencias con googletrans

In [ ]:
!pip install httpcore --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 876.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00


Añado la función para traducir los tags extraidos de las imagenes

In [20]:
import requests
import json
from translate import Translator

def translate_to_spanish(english_words):
    spanish_words = []
    for word in english_words:
        translator = Translator(to_lang="es")
        translation = translator.translate(word)
        spanish_words.append(translation)
    return spanish_words

Definimos el código que usara el API de clarifi para hacer la busqueda inversa y la extraccion de la metainformación implicita de las imagenes

In [22]:

##############################################################################
# In this section, we set the user authentication, app ID, workflow ID, and
# image URL. Change these strings to run your own example.
##############################################################################

USER_ID = 'luildosilva'
# Your PAT (Personal Access Token) can be found in the portal under Authentification
PAT = '8287b8a1de8d4b4b90ddf46011b7e737'
APP_ID = 'my-first-application'
# Change these to make your own predictions
WORKFLOW_ID = 'General'
IMAGE_URL = 'https://cloudfront-eu-central-1.images.arcpublishing.com/prisa/J32UN3N5ENDBBBC7IHYKCT2JOQ.jpg'

##########################################################################
# YOU DO NOT NEED TO CHANGE ANYTHING BELOW THIS LINE TO RUN THIS EXAMPLE
##########################################################################

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import resources_pb2, service_pb2, service_pb2_grpc
from clarifai_grpc.grpc.api.status import status_code_pb2

channel = ClarifaiChannel.get_grpc_channel()
stub = service_pb2_grpc.V2Stub(channel)

metadata = (('authorization', 'Key ' + PAT),)

userDataObject = resources_pb2.UserAppIDSet(user_id=USER_ID, app_id=APP_ID) # The userDataObject is required when using a PAT

post_workflow_results_response = stub.PostWorkflowResults(
    service_pb2.PostWorkflowResultsRequest(
        user_app_id=userDataObject,
        workflow_id=WORKFLOW_ID,
        inputs=[
            resources_pb2.Input(
                data=resources_pb2.Data(
                    image=resources_pb2.Image(
                        url=IMAGE_URL
                    )
                )
            )
        ]
    ),
    metadata=metadata
)
if post_workflow_results_response.status.code != status_code_pb2.SUCCESS:
    print(post_workflow_results_response.status)
    raise Exception("Post workflow results failed, status: " + post_workflow_results_response.status.description)

# We'll get one WorkflowResult for each input we used above. Because of one input, we have here one WorkflowResult
results = post_workflow_results_response.results[0]

# Each model we have in the workflow will produce one output.
for output in results.outputs:
    model = output.model
    print("Predicted concepts for the model `%s`" % model.id)
    english_tags = [concept.name for concept in output.data.concepts]
    spanish_tags = translate_to_spanish(english_tags)

    for en_tag, es_tag, concept in zip(english_tags, spanish_tags, output.data.concepts):
        print(f"{es_tag}: {concept.value}")

Predicted concepts for the model `general-image-recognition`
tecnología: 0.9762195944786072
Pantalla: 0.9748414158821106
pulsación: 0.971852719783783
el hombre: 0.9703907370567322
personas: 0.9654996991157532
internet: 0.9617229104042053
mujer: 0.9588173627853394
Interior: 0.9557440876960754
teléfono: 0.9536439776420593
de latón: 0.939601480960846
ordenador: 0.9370601177215576
adulto: 0.9094933271408081
familia: 0.8957862257957458
portátil: 0.8793890476226807
chica: 0.8744003772735596
inalámbrico/sin hilos: 0.8667960166931152
negocios: 0.8447944521903992
electrónica: 0.8329697251319885
retrato: 0.8254926800727844
computador personal portátil: 0.8098204731941223
Predicted concepts for the model `general-image-embedding`
Predicted concepts for the model `general-clusterering`
